In [2]:
from ultralytics.utils.custom_utils.init_setup import setup, delete_all_fiftyone_datasets

delete_all_fiftyone_datasets()
setup(0)

Loading dataset
 100% |███████████████| 6732/6732 [4.9s elapsed, 0s remaining, 1.5K samples/s]      
 100% |█████████████████| 936/936 [750.2ms elapsed, 0s remaining, 1.2K samples/s]      
 100% |███████████████| 1889/1889 [1.8s elapsed, 0s remaining, 1.0K samples/s]         
Adding transects


9557it [00:04, 1982.42it/s]


Adding dataset attributes


 86%|████████▌ | 8198/9557 [00:09<00:01, 852.57it/s] 


Could not connect session, trying again in 10 seconds



100%|██████████| 9557/9557 [00:12<00:00, 757.57it/s]



Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 s

In [1]:
from ultralytics.utils.custom_utils.helpers import get_fiftyone_dataset
import fiftyone as fo

dataset, classes = get_fiftyone_dataset(0)

sesh = fo.launch_app(dataset)

KeyboardInterrupt: 

In [6]:
import os

def create_missing_label_files(root_dir):
    # Define the root directory
    data_dir = os.path.join(root_dir, 'data', 'new_split')

    # Iterate through train, valid, and test folders
    for split in ['train', 'valid', 'test']:
        split_dir = os.path.join(data_dir, split)

        # Iterate through images folder in each split
        images_dir = os.path.join(split_dir, 'images')
        for image_file in os.listdir(images_dir):
            image_name, image_ext = os.path.splitext(image_file)
            label_file = image_name + '.txt'
            label_path = os.path.join(split_dir, 'labels', label_file)

            # Check if the label file exists, if not create an empty one
            if not os.path.exists(label_path):
                with open(label_path, 'w') as f:
                    pass  # Create an empty file

# Example usage
create_missing_label_files('/Users/vemundlund/Code/ultralytics/')


In [7]:
import os

def convert_yolo_to_bbox(yolo_annotation):
    # Parse the YOLO annotation string
    parts = yolo_annotation.split()
    class_id = int(parts[0])
    coordinates = list(map(float, parts[1:]))

    # Extract x and y coordinates
    x_coords = coordinates[::2]
    y_coords = coordinates[1::2]

    # Calculate bounding box coordinates
    x_min = min(x_coords)
    x_max = max(x_coords)
    y_min = min(y_coords)
    y_max = max(y_coords)

    # Calculate box width and height
    width = x_max - x_min
    height = y_max - y_min

    # Calculate box center
    x_center = x_min + width / 2
    y_center = y_min + height / 2

    return class_id, x_center, y_center, width, height

def convert_annotations(folder):
    for subdir, dirs, files in os.walk(folder):
        print(subdir, dirs, files)
        for file in files:
            if file.endswith(".txt"):
                file_path = os.path.join(subdir, file)
                with open(file_path, 'r') as f:
                    lines = f.readlines()
                with open(file_path, 'w') as f:
                    for line in lines:
                        class_id, x_center, y_center, width, height = convert_yolo_to_bbox(line)
                        bbox_line = f"{class_id} {x_center} {y_center} {width} {height}\n"
                        f.write(bbox_line)

# Change the directory accordingly
convert_annotations('/Users/vemundlund/Code/ultralytics/data')


/Users/vemundlund/Code/ultralytics/data ['old_split', 'new_split'] ['.DS_Store']
/Users/vemundlund/Code/ultralytics/data/old_split ['train_split', 'test_split', 'val_split', 'annet'] ['data.yaml']
/Users/vemundlund/Code/ultralytics/data/old_split/train_split ['images', 'labels'] ['.DS_Store', 'dataset.yaml']
/Users/vemundlund/Code/ultralytics/data/old_split/train_split/images ['val'] []
/Users/vemundlund/Code/ultralytics/data/old_split/train_split/images/val [] ['20230425183554_Transect 5_Sjofjaer_Liten piperenser.jpg', 'transect15_Tynn vifte_00:53:53.jpg', '20230425181954_Transect 5_Sjofjaer_Liten piperenser.jpg', 'transect04_transect04_Påvirket havbunn_00:57:44.jpg', 'transect05_transect05_Craniella spp._00:56:51.jpg', '20230428094109_Transect 6_Sjofjaer_Liten piperenser.jpg', '20230428084702_Transect 7_Sjofjaer_Liten piperenser.jpg', '20230428101604_Transect 11_Sjofjaer_Liten piperenser.jpg', '20230425183515_Transect 5_Sjofjaer_Liten piperenser.jpg', '20230428085536_Transect 7_Sjofj

In [1]:
from ultralytics.config import get_yolo_classes

print(get_yolo_classes())

[3, 5, 9, 10, 11, 15, 16, 17, 19, 20, 21, 22, 23, 28, 32]
